In [2]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-2.3.1/spark-2.3.1-bin-hadoop2.7.tgz
!tar xf spark-2.3.1-bin-hadoop2.7.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.1-bin-hadoop2.7"
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark=SparkSession.builder.getOrCreate()
df=spark.read.csv('Iris.csv',header=True,inferSchema=True)
import pyspark.sql.functions as F
from pyspark.ml.feature import StringIndexer,VectorAssembler
from pyspark.ml.classification import DecisionTreeClassifier

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ Packages [83.1 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:10 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:11 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [822 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:13 http://security.ubuntu.com/ubuntu bionic-security/restricted amd64 Packages [30.9 kB]
Get:14 http://security.ubuntu.com/ubuntu bionic-se

In [3]:
df.show(2)

+---+-------------+------------+-------------+------------+-----------+
| Id|SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|    Species|
+---+-------------+------------+-------------+------------+-----------+
|  1|          5.1|         3.5|          1.4|         0.2|Iris-setosa|
|  2|          4.9|         3.0|          1.4|         0.2|Iris-setosa|
+---+-------------+------------+-------------+------------+-----------+
only showing top 2 rows



In [4]:
df.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- SepalLengthCm: double (nullable = true)
 |-- SepalWidthCm: double (nullable = true)
 |-- PetalLengthCm: double (nullable = true)
 |-- PetalWidthCm: double (nullable = true)
 |-- Species: string (nullable = true)



# Trial 1: Raw Dataset

In [0]:
from pyspark.ml.feature import StringIndexer,VectorAssembler

In [0]:
indexer=StringIndexer(inputCol='Species',outputCol='label')
assembler=VectorAssembler(inputCols=df.columns[1:-1],outputCol='features')

In [0]:
indexer=indexer.fit(df)
df=indexer.transform(df)

In [0]:
df=assembler.transform(df)

In [0]:
training,testing=df.randomSplit([0.8,0.2],seed=666)

In [15]:
training.show(2)

+---+-------------+------------+-------------+------------+-----------+-----+-----------------+
| Id|SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|    Species|label|         features|
+---+-------------+------------+-------------+------------+-----------+-----+-----------------+
|  1|          5.1|         3.5|          1.4|         0.2|Iris-setosa|  0.0|[5.1,3.5,1.4,0.2]|
|  2|          4.9|         3.0|          1.4|         0.2|Iris-setosa|  0.0|[4.9,3.0,1.4,0.2]|
+---+-------------+------------+-------------+------------+-----------+-----+-----------------+
only showing top 2 rows



In [16]:
testing.show(2)

+---+-------------+------------+-------------+------------+-----------+-----+-----------------+
| Id|SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|    Species|label|         features|
+---+-------------+------------+-------------+------------+-----------+-----+-----------------+
|  6|          5.4|         3.9|          1.7|         0.4|Iris-setosa|  0.0|[5.4,3.9,1.7,0.4]|
| 18|          5.1|         3.5|          1.4|         0.3|Iris-setosa|  0.0|[5.1,3.5,1.4,0.3]|
+---+-------------+------------+-------------+------------+-----------+-----+-----------------+
only showing top 2 rows



In [17]:
training.count()

114

In [18]:
testing.count()

36

In [0]:
from pyspark.ml.classification import LogisticRegression

model=LogisticRegression()
model=model.fit(training)
predictions=model.transform(testing)

In [22]:
predictions.show(1)

+---+-------------+------------+-------------+------------+-----------+-----+-----------------+--------------------+-------------+----------+
| Id|SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|    Species|label|         features|       rawPrediction|  probability|prediction|
+---+-------------+------------+-------------+------------+-----------+-----+-----------------+--------------------+-------------+----------+
|  6|          5.4|         3.9|          1.7|         0.4|Iris-setosa|  0.0|[5.4,3.9,1.7,0.4]|[1314.76855309892...|[1.0,0.0,0.0]|       0.0|
+---+-------------+------------+-------------+------------+-----------+-----+-----------------+--------------------+-------------+----------+
only showing top 1 row



In [23]:
#confusion matrix
predictions.groupBy('label','prediction').count().show()

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       1.0|   12|
|  2.0|       2.0|   17|
|  1.0|       2.0|    3|
|  0.0|       0.0|    4|
+-----+----------+-----+



In [27]:
predictions.filter('prediction <> label').show(truncate=False)# misclassified values where label was 1 but it was predicted as 2

+---+-------------+------------+-------------+------------+---------------+-----+-----------------+----------------------------------------------------------+--------------------------------+----------+
|Id |SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|Species        |label|features         |rawPrediction                                             |probability                     |prediction|
+---+-------------+------------+-------------+------------+---------------+-----+-----------------+----------------------------------------------------------+--------------------------------+----------+
|69 |6.2          |2.2         |4.5          |1.5         |Iris-versicolor|1.0  |[6.2,2.2,4.5,1.5]|[-1053.0479926994533,484.76174985464445,568.2862428448091]|[0.0,5.318309292143582E-37,1.0] |2.0       |
|73 |6.3          |2.5         |4.9          |1.5         |Iris-versicolor|1.0  |[6.3,2.5,4.9,1.5]|[-922.4896673258804,430.3846856448255,492.10498168105516] |[0.0,1.5675305368975731E-27,1.

In [29]:
(predictions.filter('prediction = label').count()/predictions.count())*100 #testing accuracy

91.66666666666666

In [30]:
train_predictions=model.transform(training)
train_predictions.groupBy('label','prediction').count().show()
print((train_predictions.filter('label = prediction').count()/train_predictions.count())*100) # training accuracy

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       1.0|   35|
|  2.0|       2.0|   33|
|  0.0|       0.0|   46|
+-----+----------+-----+

100.0


# Trial 2 : With Min Max Scaling and Feature Engineering

In [31]:
# lets try feature engineering some columns
df.columns

['Id',
 'SepalLengthCm',
 'SepalWidthCm',
 'PetalLengthCm',
 'PetalWidthCm',
 'Species',
 'label',
 'features']

In [0]:
import pyspark.sql.functions as F
df=df.withColumn('SepalArea',F.round(F.col('SepalLengthCm')*F.col('SepalWidthCm'),3))
df=df.withColumn('PetalArea',F.round(F.col('PetalLengthCm')*F.col('PetalWidthCm'),3))

In [37]:
df.show(2)

+---+-------------+------------+-------------+------------+-----------+-----+-----------------+---------+---------+
| Id|SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|    Species|label|         features|SepalArea|PetalArea|
+---+-------------+------------+-------------+------------+-----------+-----+-----------------+---------+---------+
|  1|          5.1|         3.5|          1.4|         0.2|Iris-setosa|  0.0|[5.1,3.5,1.4,0.2]|    17.85|     0.28|
|  2|          4.9|         3.0|          1.4|         0.2|Iris-setosa|  0.0|[4.9,3.0,1.4,0.2]|     14.7|     0.28|
+---+-------------+------------+-------------+------------+-----------+-----+-----------------+---------+---------+
only showing top 2 rows



In [39]:
#dropping unwanted columns
df=df.drop('Id','features')
df.show(2)

+-------------+------------+-------------+------------+-----------+-----+---------+---------+
|SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|    Species|label|SepalArea|PetalArea|
+-------------+------------+-------------+------------+-----------+-----+---------+---------+
|          5.1|         3.5|          1.4|         0.2|Iris-setosa|  0.0|    17.85|     0.28|
|          4.9|         3.0|          1.4|         0.2|Iris-setosa|  0.0|     14.7|     0.28|
+-------------+------------+-------------+------------+-----------+-----+---------+---------+
only showing top 2 rows



In [40]:
df.select('Species','label').distinct().show()

+---------------+-----+
|        Species|label|
+---------------+-----+
|    Iris-setosa|  0.0|
| Iris-virginica|  2.0|
|Iris-versicolor|  1.0|
+---------------+-----+



In [41]:
df=df.drop('Species')
df.show(2)

+-------------+------------+-------------+------------+-----+---------+---------+
|SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|label|SepalArea|PetalArea|
+-------------+------------+-------------+------------+-----+---------+---------+
|          5.1|         3.5|          1.4|         0.2|  0.0|    17.85|     0.28|
|          4.9|         3.0|          1.4|         0.2|  0.0|     14.7|     0.28|
+-------------+------------+-------------+------------+-----+---------+---------+
only showing top 2 rows



In [0]:
assembler=VectorAssembler(inputCols=['SepalArea','PetalArea'],outputCol='features')
df=assembler.transform(df)

In [43]:
df.show(2)

+-------------+------------+-------------+------------+-----+---------+---------+------------+
|SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|label|SepalArea|PetalArea|    features|
+-------------+------------+-------------+------------+-----+---------+---------+------------+
|          5.1|         3.5|          1.4|         0.2|  0.0|    17.85|     0.28|[17.85,0.28]|
|          4.9|         3.0|          1.4|         0.2|  0.0|     14.7|     0.28| [14.7,0.28]|
+-------------+------------+-------------+------------+-----+---------+---------+------------+
only showing top 2 rows



In [0]:
minval=df.agg({'SepalArea':'min'}).collect()[0][0]
maxval=df.agg({'SepalArea':'max'}).collect()[0][0]

In [0]:
diff=maxval-minval
df=df.withColumn('SepalArea',(F.col('SepalArea')-minval)/diff)

In [0]:
minval=df.agg({'PetalArea':'min'}).collect()[0][0]
maxval=df.agg({'PetalArea':'max'}).collect()[0][0]
diff=maxval-minval
df=df.withColumn('PetalArea',(F.col('PetalArea')-minval)/diff)

In [53]:
df.show(2)

+-------------+------------+-------------+------------+-----+-------------------+--------------------+------------+
|SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|label|          SepalArea|           PetalArea|    features|
+-------------+------------+-------------+------------+-----+-------------------+--------------------+------------+
|          5.1|         3.5|          1.4|         0.2|  0.0| 0.3921078921078922|0.010786802030456856|[17.85,0.28]|
|          4.9|         3.0|          1.4|         0.2|  0.0|0.23476523476523473|0.010786802030456856| [14.7,0.28]|
+-------------+------------+-------------+------------+-----+-------------------+--------------------+------------+
only showing top 2 rows



In [0]:
df=df.drop('features')
assembler=VectorAssembler(inputCols=['SepalArea','PetalArea'],outputCol='features')
df=assembler.transform(df)

In [56]:
df.show(2)

+-------------+------------+-------------+------------+-----+-------------------+--------------------+--------------------+
|SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|label|          SepalArea|           PetalArea|            features|
+-------------+------------+-------------+------------+-----+-------------------+--------------------+--------------------+
|          5.1|         3.5|          1.4|         0.2|  0.0| 0.3921078921078922|0.010786802030456856|[0.39210789210789...|
|          4.9|         3.0|          1.4|         0.2|  0.0|0.23476523476523473|0.010786802030456856|[0.23476523476523...|
+-------------+------------+-------------+------------+-----+-------------------+--------------------+--------------------+
only showing top 2 rows



In [64]:
training,testing=df.randomSplit([0.8,0.2],seed=666)
model=LogisticRegression()
model=model.fit(training)
test_preds=model.transform(testing)
print('*'*10)
print('Testing Summary')
print('*'*10)
test_preds.groupBy('label','prediction').count().show()
accuracy=(test_preds.filter('label = prediction').count())/(test_preds.count())
print(f' Accuracy : {round(accuracy,2)*100}%')
print('\n')
train_preds=model.transform(training)
print('*'*10)
print('Training Summary')
print('*'*10)
train_preds.groupBy('label','prediction').count().show()
accuracy=(train_preds.filter('label = prediction').count())/(train_preds.count())
print(f' Accuracy : {round(accuracy*100,2)}%')# result after min max scaling and feature engineering

**********
Testing Summary
**********
+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       1.0|   14|
|  2.0|       2.0|   17|
|  1.0|       2.0|    1|
|  0.0|       0.0|    4|
+-----+----------+-----+

 Accuracy : 97.0%


**********
Training Summary
**********
+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       1.0|   33|
|  2.0|       2.0|   32|
|  2.0|       1.0|    1|
|  1.0|       2.0|    2|
|  0.0|       0.0|   46|
+-----+----------+-----+

 Accuracy : 97.37%
